# Graph Construction

In this notebook, we are exploring the standard construction of the Graph using a GNN + GSL model.

In [1]:
from config import CONFIG
from model.metro_model import MetroModel
from data.metro_dataset import MetroDataset

import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
import os
from pathlib import Path
import networkx as nx
import numpy as np

torch.manual_seed(0)

c:\Users\simon\Projets\CountryTrends\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\simon\Projets\CountryTrends\.conda\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
c:\Users\simon\Projets\CountryTrends\.conda\lib\site-packages\momepy\coins.py:15: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
imp

In [2]:
dataset = MetroDataset(CONFIG.lines, init_nb=20)
num_nodes = dataset.cg.num_nodes()
dataset.cg.altair_graph(dataset.cg.graph, 10)

c:\Users\simon\Projets\CountryTrends\.conda\lib\site-packages\momepy\utils.py:430: UserWarning: Approach is not set. Defaulting to 'primal'.
  warnings.warn("Approach is not set. Defaulting to 'primal'.")


alt.LayerChart(...)

## Model

In [3]:
model = MetroModel(embedding_size=8, num_nodes=num_nodes, neighbor_nb=1, input_size=1, gsl_mode="matrix")

## Training

In [4]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=8)
checkpoint_callback = ModelCheckpoint(dirpath="checkpoints/", save_top_k=1000 ,monitor="loss") # , every_n_train_steps=100, 
trainer = Trainer(max_epochs=50, callbacks=[checkpoint_callback])
trainer.fit(model, train_loader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\simon\Projets\CountryTrends\.conda\lib\site-packages\pytorch_lightning\trainer\setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
c:\Users\simon\Projets\CountryTrends\.conda\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
c:\Users\simon\Pro

Epoch 49: 100%|██████████| 248/248 [00:01<00:00, 212.82it/s, loss=17, v_num=0]     

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 248/248 [00:01<00:00, 211.37it/s, loss=17, v_num=0]


## Visualization of results

In [5]:
def load_model(path):
    checkpoint = torch.load(path)
    model = MetroModel(embedding_size=8, num_nodes=num_nodes, neighbor_nb=1, input_size=1, gsl_mode="matrix")
    model.load_state_dict(checkpoint['state_dict'])
    return model

def viz(checkpoint):
    model = load_model(checkpoint)

    A = model.graph_matrix_learning().detach().numpy()
    A = np.array(A)

    learned_graph = nx.from_numpy_array(A, nx.DiGraph)
    return dataset.cg.altair_graph(learned_graph, 10)

for path in os.listdir('checkpoints'):
    checkpoint = Path('checkpoints') / path
    print(checkpoint)

checkpoints\epoch=0-step=248-v1.ckpt
checkpoints\epoch=0-step=248.ckpt
checkpoints\epoch=1-step=496-v1.ckpt
checkpoints\epoch=1-step=496.ckpt
checkpoints\epoch=10-step=2728-v1.ckpt
checkpoints\epoch=10-step=2728.ckpt
checkpoints\epoch=11-step=2976-v1.ckpt
checkpoints\epoch=11-step=2976.ckpt
checkpoints\epoch=12-step=3224-v1.ckpt
checkpoints\epoch=12-step=3224.ckpt
checkpoints\epoch=13-step=3472-v1.ckpt
checkpoints\epoch=13-step=3472.ckpt
checkpoints\epoch=14-step=3720-v1.ckpt
checkpoints\epoch=14-step=3720.ckpt
checkpoints\epoch=15-step=3968-v1.ckpt
checkpoints\epoch=15-step=3968.ckpt
checkpoints\epoch=16-step=4216-v1.ckpt
checkpoints\epoch=16-step=4216.ckpt
checkpoints\epoch=17-step=4464-v1.ckpt
checkpoints\epoch=17-step=4464.ckpt
checkpoints\epoch=18-step=4712-v1.ckpt
checkpoints\epoch=18-step=4712.ckpt
checkpoints\epoch=19-step=4960-v1.ckpt
checkpoints\epoch=19-step=4960.ckpt
checkpoints\epoch=2-step=744-v1.ckpt
checkpoints\epoch=2-step=744.ckpt
checkpoints\epoch=20-step=5208-v1.ckp

In [7]:
viz("checkpoints\epoch=30-step=7688-v1.ckpt")

alt.LayerChart(...)